<h1><center>Learning TensorFlow Basics </center></h1>

## Concepts:

    1. Computational graph
    2. Nodes in graph are madeup of tensors
    3. Tensors are multidimensional arrays and basic datatype of TF
    4. Graph should be built by declaring and connecting tensor nodes
    5. Graph/ node should be evaluated by running in a session

## Datatypes:
    1. constant()
    2. placeholder()
    3. Variable()
    
## Core training schemas:
    1. Optimizers
    2. Setup model, then setup loss function, minimize it > Variables get tuned
    
## tf.contrib.learn
    Built-in classes for out-of-box implementation > Eliminates `Setup model` part
    
    

In [2]:
import tensorflow as tf

# constant() and placeholder() example

const_node = tf.constant(3.0, dtype=tf.float32)  # A tensor of shape 0
print const_node
const_node_1 = tf.constant([[3.0, 9.0, 33.0]], dtype=tf.float32)  # A tensor of shape 1
print const_node_1

# Session concept
with tf.Session() as sess:
    print sess.run(const_node_1 + const_node)  # Should result in error?
    print sess.run(const_node_1 * const_node)  # Should result in error?
    print sess.run(const_node_1 + 34)  # Is this boxing int 34 to respective tensorflow Constant int class?
    print sess.run(tf.one_hot(indices=[1], depth=5))

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(1, 3), dtype=float32)
[[  6.  12.  36.]]
[[  9.  27.  99.]]
[[ 37.  43.  67.]]
[[ 0.  1.  0.  0.  0.]]


In [22]:
# Variable() example.
# constant() will be initialized immediately after declarations and their node values can't change in runtime.
# But Variables() values can. They have to be initialized seperately by running a special node

# Linear model
W = tf.Variable([-10], dtype=tf.float32)  # Shape 1
x = tf.placeholder(tf.float32)            # Empty, assertion that says we supply variables later
b = tf.Variable([-13], dtype=tf.float32)  # Shape 1
b0 = tf.Variable(-13, dtype=tf.float32)   # Shape 0
linear_model = W * x + b
linear_model0 = W * x + b0
# Shapes should be compatable! 
print W.shape

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(linear_model, {x: [1, 2, 3, 4]})
    print sess.run(linear_model0, {x: [1, 2, 3, 4]})
    
    
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print "------------------current section---------------------"
# Loss function for linear model
y = tf.placeholder(tf.float32)
loss = tf.reduce_sum(tf.square(linear_model - y))
print sess.run(loss, {x: [1, 2, 3, 4], y: [-1, -2, -3, -4]})



(1,)
[-23. -33. -43. -53.]
[-23. -33. -43. -53.]
------------------current section---------------------
5446.0


In [27]:
# Training a model
optimizer = tf.train.GradientDescentOptimizer(0.01)  # GD optimizer meat
train_node = optimizer.minimize(loss)                # A node with inputs and params linked
for _ in range(100):
    sess.run(train_node, {x: [1, 2, 3, 4], y: [1, 2, 3, 4]})  # `loss` involves place holders, supply them

print sess.run([W, b, loss], {x: [1, 2, 3, 4], y: [1, 2, 3, 4]})


[array([ 1.00001621], dtype=float32), array([ -4.75694615e-05], dtype=float32), 1.5077433e-09]


   
    Use tf.learn to build models out of box with inbuilt types like LinearRegressor, LogisticRegressor etc.. 
    All these built-in models are sub-classed from tf.learn.Estimator class.
    Use it to define your own models which takes advantage of all the abstraction in tf.learn 


# Exploring MNIST Dataset

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print mnist.train.num_examples, mnist.test.num_examples, mnist.validation.num_examples
print mnist.train.images.shape

batch = mnist.train.next_batch(2)
print batch[0][0].shape
import math; print math.sqrt(784) # 28 X 28 X 1 Input image

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
55000 10000 5000
(55000, 784)
(784,)
28.0


In [33]:
# We will be using a softmax regression as our model which learns 0-9 digits

# Inputs 
x = tf.placeholder(tf.float32, [None, 784])

# Trainable vars
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Model 
# y = softmax(Wx + b)
y = tf.nn.softmax(tf.matmul(x, W) + b)

# Loss fn.
y_ = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Minimizing loss fn.
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for _ in range(1000):
    xs, ys = mnist.train.next_batch(100)
    sess.run(train, {x: xs, y_: ys})
print "Finished training..."

Finished training...


In [44]:
# Model evaluation...

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Prediction accuracy : %.2f percent" 
      % (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}) * 100))


Prediction accuracy : 91.37 percent


___

# Convolution Neural Network

## Beauty
### One line 
Map a fully lookable image to a vector of features

### Summary
Input image is say 512 X 512 X 3, it will be convolved, CONVOLUTION LAYER (learns a set of kernal matrices which will pick key features) to new volumes. These new volumes will be sampled-down to lower volumes, POOLING LAYER. These will, after certain depth learn to identify proper high-level features. Then we simply use these features on a FULLY-CONNECTED LAYER to learn and predict classes.

### Quick start
Gaining momentum on Convolution Neural Network via https://www.youtube.com/watch?v=LxfUGhug-iQ&list=PLwQyV9I_3POsyBPRNUU_ryNfXzgfkiw2p&index=7


# Applying CNN on a MNIST

In [5]:
# Generating random numbers to initialize variables.
values =  tf.truncated_normal([4,4], stddev=1, mean=10)
sess = tf.Session()
sess.run(values)

array([[  9.51897144,  10.48738194,  11.74557304,   8.70279503],
       [ 10.0640316 ,   8.57131863,   9.68689442,  10.04261208],
       [  8.95930099,  10.03546906,   9.84476662,   9.6274662 ],
       [  9.76712513,  10.90784836,  10.4563446 ,   9.48399067]], dtype=float32)

## Model description

 - Input images are of shape (28, 28, 1)
 - We shall have 2 layers deep network 
 - First layer shall take (28, 28, 1) and return (28, 28, 32); since we plan to use 32 (5, 5) filters 
 - First layer shall use (2, 2) MAX-POOL matrices and recude (28, 28, 32) to (14, 14, 32)
 - Second layer shall take (14, 14, 32) and apply 64 (5, 5) kernels [Or semantically 2 (5, 5, 32)?]
 - Second conv layer then shall output (14, 14, 64) tensor which will be passed on to MAX-POOL layer 2
 - Second max pool layer will again use (2, 2) with stride 2 to reduce its input to (7, 7, 64)
 - Then we flatten this output to 7 * 7 * 64 vector and input it to a FULLY CONNECTED layer 
 

In [2]:
import tensorflow as tf

# Initializing Weights of kernels 
def get_weights_var(shape):
    
    weights_matrix = tf.truncated_normal(shape=shape, stddev=0.1)
    
    return tf.Variable(weights_matrix)

# Initializing Biases to use in ReLU output
def get_bias_var(shape):
    
    bias_vect = tf.truncated_normal(shape, stddev=0.1)
    
    return tf.Variable(bias_vect)

# Return a typical Conv layer operation
def conv_2d(x, W):
    
    return tf.nn.conv2d(input=x, filter=W, padding="SAME", strides=[1, 1, 1, 1])

# Return a typical Pool layer operation
def pool_2x2(x):
    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

## Start building the graph

# First conv layer
W1 = get_weights_var([5, 5, 1, 32])
b1 = get_bias_var([32])

## Reshaping inputs from 784 -> 28x28
inputs, labels = mnist.train.next_batch(50)  # We will use labels while training...
print inputs.shape
x_ = inputs.reshape([-1, 28, 28, 1])
print x_.shape

## But let's just give x as a place holder as of now...
x = tf.placeholder(tf.float32, shape=[None, 784])
x_images = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_2d(x_images, W=W1)
relu1 = tf.nn.relu(conv1 + b1)
pool1 = pool_2x2(relu1)


# Second conv layer
W2 = get_weights_var([5, 5, 32, 64])
b2 = get_bias_var([64])

conv2 = conv_2d(x=pool1, W=W2)
relu2 = tf.nn.relu(conv2 + b2)
pool2 = pool_2x2(relu2)

# Fully connected layer 

## First flatten out output from pool2
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

## Now develop vars for fully connected layer
fc1_w = get_weights_var([7 * 7 * 64, 1024])
fc1_b = get_bias_var([1024])
fc1 = tf.nn.relu(tf.matmul(pool2_flat, fc1_w) + fc1_b)

# And finally output layer
out_w = get_weights_var([1024, 10])
out_b = get_bias_var([10])
out = tf.matmul(fc1 , out_w) + out_b

##############################################################

# Uptil here, the graphical model is built 
# Let's now define metric to esitmate loss and define optimizers to minimize it (TRAINING)
y_ = tf.placeholder(dtype=tf.float32)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=out))
train = tf.train.AdamOptimizer(1e-04).minimize(loss)


##############################################################

# Metrics to measure accuracy of trained model
correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

##############################################################
# Now the actual training...

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1]})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run(train, feed_dict={x: batch[0], y_: batch[1]})
    
print "###############################################################"
print "#                    TRAINING COMPLETE                        #"
print "###############################################################"


        

(50, 784)
(50, 28, 28, 1)
step 0, training accuracy 0.06
step 100, training accuracy 0.8
step 200, training accuracy 0.98
step 300, training accuracy 0.96
step 400, training accuracy 0.96
step 500, training accuracy 0.96
step 600, training accuracy 1
step 700, training accuracy 0.98
step 800, training accuracy 0.98
step 900, training accuracy 0.94
step 1000, training accuracy 0.92
step 1100, training accuracy 0.96
step 1200, training accuracy 0.98
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 1
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 0.98
step 2600, training accuracy 1
step 2700, training accuracy 0.98
step 2800, training accuracy 0.98
step 2900, training ac

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [9]:
print('Test accuracy %g percent' % (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}) * 100))

Test accuracy 98.93 percent


<h2><center>Happy Ending! </center></h2>